In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

import pandas as pd

from py_files.datasets import GetRepsDataset

from tqdm.notebook import trange, tqdm

import numpy as np

In [2]:
class Resnet18backbone(nn.Module):
    def __init__(self):
        super(Resnet18backbone, self).__init__()
        
        self.resnet_head = nn.Sequential(*list(models.resnet18(pretrained=True).children())[:-1])
        
    def forward(self, x):
        out = self.resnet_head(x)
        return out

In [3]:
# test
model = Resnet18backbone()

normalize = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
batch_size = 64

transform=transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ConvertImageDtype(torch.float),
    normalize,
        ])

df = pd.read_csv('../csv/working_df.csv')

train_dataset = GetRepsDataset(df, 'train', transform)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=64, shuffle=False, num_workers=1, pin_memory=True)

In [4]:
model(train_loader.dataset.__getitem__(1)[0].unsqueeze(0)).shape

torch.Size([1, 512, 1, 1])

In [5]:
def check_cuda():
    if torch.cuda.is_available():       
        device = torch.device("cuda")
        print(f'There are {torch.cuda.device_count()} GPU(s) available.')
        print('Device name:', torch.cuda.get_device_name(0))
        return device
    else:
        print('No GPU available, using the CPU instead.')
        device = torch.device("cpu")
        return device
    
device=check_cuda()

No GPU available, using the CPU instead.


In [17]:
model = model.to(device)

rep_list = []
path_list = []
model.eval()
for img, path in tqdm(train_loader):
    
    img = img.to(device)    
    
    in_batch_size = img.shape[0]
    
    reps = model(img)
    rep_list.append(reps.detach().detach().cpu().numpy().reshape(in_batch_size, -1))
    path_list += path
    
rep_array = np.concatenate(rep_list, axis=0)
df = pd.DataFrame(rep_array)
df['patch_paths']=path_list

df.to_csv('../csv/working_train_reps.csv', index=False)

  0%|          | 0/1702 [00:00<?, ?it/s]

In [16]:
rep_array = np.concatenate(rep_list, axis=0)
df = pd.DataFrame(rep_array)
df['patch_paths']=path_list

df.to_csv('../csv/working_train_reps.csv', index=False)